In [13]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
import csv

# Mostrar el directorio actual
print(os.getcwd())

# Cambiar el directorio de trabajo
os.chdir('/Users/matdknu/Dropbox/social-data-science/wiki-chile_project')



/Users/matdknu/Dropbox/social-data-science/wiki-chile_project


In [14]:
import pandas as pd

# Leer la hoja llamada 'familias_argentinas'
df_wikipedia = pd.read_excel('data/raw_data/familia_link_manual2.xlsx', sheet_name="familias_argentinas")

# Mostrar las primeras filas
print("Wikipedia:")
print(df_wikipedia.head())




Wikipedia:
                                                 url
0  https://es.wikipedia.org/wiki/Fernando_Abal_Me...
1  https://es.wikipedia.org/wiki/Juan_Manuel_Abal...
2     https://es.wikipedia.org/wiki/Nilda_Garr%C3%A9
3  https://es.wikipedia.org/wiki/Juan_Manuel_Abal...
4  https://es.wikipedia.org/wiki/%C3%81lvaro_Jos%...


In [15]:
# Normalizar nombres de columnas
df_wikipedia.columns = df_wikipedia.columns.str.strip().str.lower()

# Extraer links únicos (suponiendo que la columna se llama 'link')
initial_urls = df_wikipedia['url'].dropna().unique().tolist()

# Mostrar resultado formateado para pegar
print("initial_urls = [")
for url in initial_urls:
    print(f'    "{url}",')
print("]")


initial_urls = [
    "https://es.wikipedia.org/wiki/Fernando_Abal_Medina",
    "https://es.wikipedia.org/wiki/Juan_Manuel_Abal_Medina",
    "https://es.wikipedia.org/wiki/Nilda_Garr%C3%A9",
    "https://es.wikipedia.org/wiki/Juan_Manuel_Abal_Medina_(hijo)",
    "https://es.wikipedia.org/wiki/%C3%81lvaro_Jos%C3%A9_de_Alzogaray",
    "https://es.wikipedia.org/wiki/%C3%81lvaro_Alsogaray",
    "https://es.wikipedia.org/wiki/Mar%C3%ADa_Julia_Alsogaray",
    "https://es.wikipedia.org/wiki/Julio_Rodolfo_Alsogaray",
    "https://es.wikipedia.org/wiki/%C3%81lvaro_Alzogaray",
    "https://es.wikipedia.org/wiki/Adriana_Cecilia_Barcia",
    "https://es.wikipedia.org/wiki/Jos%C3%A9_Manuel_Astigueta_(hijo)",
    "https://es.wikipedia.org/wiki/Jos%C3%A9_Mariano_Astigueta_(nieto)",
    "https://es.wikipedia.org/wiki/Jos%C3%A9_Mariano_Astigueta_(padre)",
    "https://es.wikipedia.org/wiki/Jos%C3%A9_Manuel_Astigueta",
    "https://es.wikipedia.org/wiki/Jos%C3%A9_Manuel_Astigueta_(padre)",
    "https://e

In [16]:
CSV_FILENAME = "datos_argentina_manual2.csv"
BASE_URL = "https://es.wikipedia.org"


visited_urls = set()

# Función para extraer información de una persona o entidad
def extract_info(url, depth=0, max_depth=2):
    if url in visited_urls or depth > max_depth:
        return None
    visited_urls.add(url)

    response = requests.get(url)
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    infobox = soup.find("table", class_="infobox biography vcard")
    if not infobox:
        return None

    info = {
        "Nombre": "",
        "Fecha de nacimiento": "",
        "Residencia": "",
        "Nacionalidad": "",
        "Cargos": "",
        "Educación": "",
        "Trabajos previos": "",
        "Área": "",
        "Partido político": "",
        "Familia": "",
        "URL": url
    }

    title = soup.find("h1", class_="firstHeading")
    if title:
        info["Nombre"] = title.text.strip()

    for row in infobox.find_all("tr"):
        header = row.find("th")
        value = row.find("td")
        if header and value:
            header_text = header.text.strip()
            value_text = value.text.strip()

            if "Nacimiento" in header_text:
                info["Fecha de nacimiento"] = value_text
            elif "Residencia" in header_text:
                info["Residencia"] = value_text
            elif "Nacionalidad" in header_text:
                info["Nacionalidad"] = value_text
            elif any(k in header_text for k in ["Ocupación", "Cargos", "Cargo"]):
                for item in value.stripped_strings:
                    info["Cargos"] += item + "; "
            elif any(k in header_text for k in ["Educación", "Educado en", "Educada en"]):
                education_links = [link.text for link in value.find_all("a") if link.get("href", "").startswith("/wiki/")]
                info["Educación"] = "; ".join(education_links)
            elif "Área" in header_text:
                info["Área"] = value_text
            elif "Partido político" in header_text:
                parties = [link.text for link in value.find_all("a")]
                info["Partido político"] = "; ".join(parties)
            elif any(k in header_text for k in ["Padres", "Cónyuge", "Hijos", "Familia"]):
                family_links = []
                for link in value.find_all("a"):
                    href = link.get("href", "")
                    if (href.startswith("/wiki/") and
                        not any(href.startswith(excl) for excl in [
                            "/wiki/Ayuda:", "/wiki/Archivo:", "/wiki/Especial:", "/wiki/Plantilla:", "/wiki/Portal:", "/wiki/Categor%C3%ADa:", "/wiki/Familia_"])):
                        family_links.append(f"{link.text} ({BASE_URL}{href})")
                    else:
                        family_links.append(link.text)
                info["Familia"] += "; ".join(family_links) + "; "

    # Extraer trabajos previos desde bloques válidos únicamente
    rows = infobox.find_all("tr")
    for i in range(len(rows) - 1):
        th = rows[i].find("th")
        td = rows[i + 1].find("td")
        if th and td and th.get("colspan") == "3" and 'background-color:#E6E6FA' in th.get("style", ""):
            if th.find("a"):  # asegurarse que tiene enlace, para evitar otros bloques como altura, religión, etc.
                cargos = [a.text for a in th.find_all("a") if a.get("href", "").startswith("/wiki/")]
                fechas = td.get_text(strip=True)
                cargo_str = "; ".join(cargos)
                info["Trabajos previos"] += f"{cargo_str} – {fechas}; "

    save_to_csv(info)
    print(f"Guardado: {info['Nombre']}")

    for family_member in info["Familia"].split("; "):
        if "(" in family_member and ")" in family_member:
            family_url = family_member.split("(")[1].split(")")[0]
            if ("wikipedia" in family_url and not "Familia_" in family_url):
                extract_info(family_url, depth + 1, max_depth)
                time.sleep(2)

# Guardar datos en CSV
def save_to_csv(data):
    with open(CSV_FILENAME, "a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data.keys())
        if file.tell() == 0:
            writer.writeheader()
        writer.writerow(data)

# Limpiar archivo CSV al inicio
open(CSV_FILENAME, "w", encoding="utf-8").close()

# Iniciar scraping
for url in initial_urls:
    extract_info(url)
    time.sleep(2)

# Eliminar duplicados
df = pd.read_csv(CSV_FILENAME)
df.drop_duplicates(subset=["Nombre"], keep="first", inplace=True)
df.to_csv(CSV_FILENAME, index=False)
print("Proceso completado. Datos guardados en", CSV_FILENAME)


Guardado: Fernando Abal Medina
Guardado: Juan Manuel Abal Medina
Guardado: Nilda Garré
Guardado: Juan Manuel Abal Medina
Guardado: Juan Manuel Abal Medina (hijo)
Guardado: Álvaro José de Alzogaray
Guardado: Álvaro Alsogaray
Guardado: María Julia Alsogaray
Guardado: Julio Rodolfo Alsogaray
Guardado: Juan Carlos Alsogaray
Guardado: Álvaro José de Alzogaray
Guardado: Adriana Cecilia Barcia
Guardado: José Manuel Astigueta (hijo)
Guardado: José Mariano Astigueta (nieto)
Guardado: José Mariano Astigueta (padre)
Guardado: José Manuel Astigueta (padre)
Guardado: José Manuel Astigueta (padre)
Guardado: José Manuel Astigueta (padre)
Guardado: Silvina Bullrich
Guardado: Arturo Bullrich
Guardado: Luisa Vehil
Guardado: Miguel Ángel Solá
Guardado: Rodolfo Galimberti
Guardado: Patricia Bullrich
Guardado: Esteban Bullrich
Guardado: Adolfo Bullrich
Guardado: Fabiana Cantilo
Guardado: Alfredo Francisco Cantilo
Guardado: Augusto Bullrich
Guardado: Francisco Bullrich
Guardado: Alicia Cazzaniga
Guardado: J